### zeptat se vavrince:

- PD trigger 15v/5v -> step-up -> bat_protection_board gut?
- je neco takovehohle pouzitelne? https://www.gme.cz/v/1508436/led-indikator-stavu-li-ion-pol-baterie-4s
- baterky -> +-15v zdroj pres 5v, nebo z baterek (ty jsou nekdy nad 15v nekdy pod, tak u boostu asi nicmoc)

# IN PROGRESS:
- accept infos in UI

## TODO:
- add batery indicators 
  - 4led https://www.gme.cz/v/1508436/led-indikator-stavu-li-ion-pol-baterie-4s <!-- https://pajenicko.cz/indikator-stavu-kapacity-lipol-liion-baterie-4s -->
  - voltage https://www.postavrobota.cz/LED-Indikator-stavu-kapacity-baterie-do-panelu-4S-li-ion-li-po-s-voltmetrem-d655.htm
  - nejlip adc na rpi
- rewrite wifi_manager to change what configs services use instead of changing configs themselves
- sockets in wifi_man
- sockets use envelopes to separate wifiman and lasershow -> mention in thesis
- sockets in web_ui, discord_bot
- rewrite discord_bot

## thesis uvod
laser scanning se využívá v různých oborech víc a víc, cílem me práce je umožnit začínajícím kutilum se seznámit s touto technologií a rozvíjet se s ní

### menu.hpp

  // prusa style ui with oled display and rotary encoder - use ZMQ sockets for communication with lasershow.cpp


### discord help command

### wifi manager
sockets

### web-ui
sockets


## hw_notes:
18650 libovolny pocet paralelne
nabijeni usb-c https://www.laskakit.cz/nabijecka-li-ion-clanku-tp4056-s-ochranou-usb-c/#relatedFiles

z baterek:
	symetricky stepup +-15v signal galva https://www.laskakit.cz/step-up-boost-menic-s-xl6007-15v-zdroj-symetrickeho-napeti/
	5v stepup RPI, cipy, OLED, ...
	15v 1A stepup zdroj galva
	snad nebude potřeba -15v, na puvodnim zdroji je +15 1A a -15 0.5A
	12v 1A stepup zdroj lase

jak undervoltage protection?

## sw_notes:
lasershow receives shutdown message and cleans up laser - turn off relay
  also measure battery voltage

//how do i debounce well??

## WATCH-OUT
hostapd
  - The previously supported configuration setting DAEMON_CONF in
  /etc/default/hostapd is deprecated and its support will be removed.

  - SCRIPT DEPRECATION WARNING                    
  This script, located at https://deb.nodesource.com/setup_X, used to
  install Node.js is deprecated now and will eventually be made inactive.
  Please visit the NodeSource distributions Github and follow the
  instructions to migrate your repo.
  https://github.com/nodesource/distributions
  The NodeSource Node.js Linux distributions GitHub repository contains
  information about which versions of Node.js and which Linux distributions
  are supported and how to install it.
  https://github.com/nodesource/distributions
  https://github.com/nodesource/distributions#debian-versions


In [ ]:
total = 0

filament = 500
# max 500czk za jeden - maybe 2x  
filament *= 2
total += filament

RPi = 2200 # RPi 5 8GB (wow, banger)
total += RPi

galva = 2200
total += galva

laser = 1800
total += laser

signal = 0
signal += 90
signal += 2* 50 # amps - 30-50czk
total += signal

napajeni = 0
# baterky
napajeni += 4*275 # 4-6x (podle toho, kolik se jich vleze) 18650 idk kolik stoji
# 4000mAh 275kc https://www.abaterie.cz/baterie-samsung-21700-inr21700-40t-li-ion-3-6-v-4000-mah/
# 2x 18650 - 6000mAh 425kc https://www.abaterie.cz/baterie-samsung-2xinr18650-30q-6000mah-20a/

#ochrana
napajeni += 58 #https://www.laskakit.cz/ochrana-li-ion-baterie-4s-40a/

# nabijeni
napajeni += 48 # usb-c PD trigger board https://www.laskakit.cz/usb-c-pd-qc-prepinac-napajeciho-napeti/
napajeni += 38 # 4A step-up https://www.laskakit.cz/step-up-boost-menic-s-xl6009--modra/

# napeti
napajeni += 48 # 3A RPi https://www.laskakit.cz/step-down-menic-s-xl4005/
napajeni += 28 # 1A galva https://www.laskakit.cz/step-down-menic-s-lm2596/
napajeni += 28 # 1A laser https://www.laskakit.cz/step-down-menic-s-lm2596/
napajeni += 88 # +-15V boost https://www.laskakit.cz/step-up-boost-menic-s-xl6007-15v-zdroj-symetrickeho-napeti/
# +-15V boost connected to rpis 5V
napajeni += 70 # 65kc balikovna

# deprecated - kdyby paralelne -> step-ups
# napajeni += 38 # 3A RPi https://www.laskakit.cz/step-up-boost-menic-s-xl6009--modra/
# napajeni += 18 # 2A galva 
# napajeni += 18 # 2A laser (1A would be enough) 
# napajeni += 88 # +-15V boost https://www.laskakit.cz/step-up-boost-menic-s-xl6007-15v-zdroj-symetrickeho-napeti/
# napajeni += 70 # 65kc balikovna

total += napajeni

oled = 160 # https://www.laskakit.cz/20x4-lcd-displej-2004-modry-i2c-prevodnik/
total += oled

print ("mone: " + str(total))

SyntaxError: invalid decimal literal (3591834150.py, line 43)

In [ ]:
Vbat = 14.8 # 4x 18650 (3.7*4 = 14.8V, 4.2*4 = 16.8V)
Imax = 40 # 10A per 18650 is less than the max 35A

Pmax = Vbat * Imax

print ("max power: " + str(Pmax) + "W")

P = 0
# 3A RPi
P += 5 * 3 # 5V
# -15V boost -> signal galva

# 1A galva
P += 1 * 15 # 15V
# 1A laser
P += 1 * 12 # 12V

print ("needed power: " + str(P) + "W")